<a href="https://colab.research.google.com/github/markl-a/LLM-agent-Demo/blob/main/1.LangchainDemo/1.langchain官網使用範例：RAG問答/Q%26A_with_RAG_簡介以及快速開始.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

LLM應用中幾個較為有的應用當數複雜的問答 (Q&A) 聊天機器人。這些應用程式可以回答有關特定來源資訊的問題。這些應用程式使用一種稱為檢索增強生成（RAG）的技術。

# 什麼是 RAG

RAG 是一種利用額外資料來增強 LLM 知識的技術。

LLM可以用來回應廣泛的主題，但他們的知識僅限於他們接受訓練的的數據。如果您想要建立能夠推理私有資料或模型截止日期之後引入的資料的 AI 應用程序，您需要使用模型所需的特定資訊來增強模型的知識。引入適當的資訊並將其插入模型提示的過程稱為檢索增強生成 (RAG)。

LangChain 有許多元件旨在幫助建立問答應用程式以及更廣泛的 RAG 應用程式。

注意：這裡我們將重點放在非結構化資料的問答。我們在其他地方介紹的兩個 RAG 用例是：

1.  [SQL資料問答](https://python.langchain.com/docs/use_cases/sql/)
2.  [程式碼問答（例如 Python）](https://python.langchain.com/docs/use_cases/code_understanding/)

# RAG Architecture

典型的 RAG 應用程式有兩個主要組件：

索引：用於從來源取得資料並為其建立索引的管道。 這通常發生在離線狀態。

檢索和生成：實際的 RAG 鏈，它在運行時接受用戶查詢並從索引中檢索相關數據，然後將其傳遞給模型。

從原始資料到答案的最常見的完整序列如下所示：

索引

1.   載入：首先我們需要載入資料。 這是透過[DocumentLoaders](https://python.langchain.com/docs/use_cases/question_answering/#:~:text=is%20done%20with-,DocumentLoaders,-.) 完成的。
2.   分割：[文字分割器](https://python.langchain.com/docs/use_cases/question_answering/#:~:text=Split%3A-,Text%20splitters,-break%20large%20Documents)將大文檔分成更小的區塊。 這對於索引資料和將其傳遞到模型都很有用，因為大塊更難搜尋並且不適合模型的有限上下文視窗。
3.    儲存：我們需要某個地方來儲存和索引我們的分割，以便以後可以搜尋它們。 這通常是使用 [VectorStore](https://python.langchain.com/docs/modules/data_connection/vectorstores/) 和 [Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/) 模型來完成的。




![範例1](https://raw.githubusercontent.com/markl-a/LLM-agent-Demo/main/1.LangchainDemo/1.langchain%E5%AE%98%E7%B6%B2%E4%BD%BF%E7%94%A8%E7%AF%84%E4%BE%8B%EF%BC%9ARAG%E5%95%8F%E7%AD%94/rag_indexing-1.png)

檢索和生成


1.   檢索：給定使用者輸入，使用[檢索器](https://python.langchain.com/docs/modules/data_connection/retrievers/)從儲存中檢索相關分割。
2.   產生：[ChatModel](https://python.langchain.com/docs/modules/model_io/chat/) / [LLM](https://python.langchain.com/docs/modules/model_io/llms/) 使用包含問題和檢索到的資料的提示產生答案





![範例2](https://raw.githubusercontent.com/markl-a/LLM-agent-Demo/main/1.LangchainDemo/1.langchain官網使用範例：RAG問答/rag_retrieval_generation-2.png)

# 目錄



*   [快速入門 Quickstart](https://python.langchain.com/docs/use_cases/question_answering/quickstart/)：我們建議從這裡開始。 以下許多指南假設您完全理解快速入門中所示的體系結構。
*   [傳回來源 Returning sources](https://python.langchain.com/docs/use_cases/question_answering/sources/)：如何傳回特定生成方法中使用的來源文件。
*   [串流傳輸 Streaming](https://python.langchain.com/docs/use_cases/question_answering/streaming/)：如何串流最終答案以及中間步驟。
*   [新增聊天記錄 Adding chat history](https://python.langchain.com/docs/use_cases/question_answering/chat_history/)：如何將聊天記錄新增至問答應用程式。
*   [按使用者檢索 Per-user retrieval](https://python.langchain.com/docs/use_cases/question_answering/per_user/)：每個使用者都有自己的私有資料時如何進行檢索。
*   [使用代理 Using agents](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents/)：如何使用代理進行問答。
*   [使用本地模型 Using local models](https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa/)：如何使用本地模型進行問答。








# Quickstart 快速開始

LangChain 有許多元件旨在幫助建立問答應用程式以及更廣泛的 RAG 應用程式。為了熟悉這些，我們將基於文字資料來源建立一個簡單的問答應用程式。在此過程中，我們將跑過一個典型的問答架構一遍，並介紹相關的 LangChain 元件，並重點關注於進階問答技術的其他資源。我們還將看到 LangSmith 如何幫助我們追蹤和理解我們的應用程式。隨著我們的應用程式變得越來越複雜，LangSmith 將變得越來越有幫助。

## Architecture 架構

我們將創建一個典型的 RAG 應用程序，如[問答簡介](https://python.langchain.com/docs/use_cases/question_answering/)中所述，它有兩個主要組件：

* 索引：用於從來源取得資料並為其建立索引的流水線。這通常在離線狀態運。

* 檢索和生成：實際的 RAG 鏈，它在運行時接受用戶查詢並從索引中檢索相關數據，然後將其傳遞給模型。

從原始資料到答案的完整序列如下所示：

### Indexing 索引

* 載入：首先我們需要載入資料。 為此，我們將使用 [DocumentLoaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/) 處理資料 。

* 分割：[文字分割器Text splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)  將大文字分割Documents成更小的文字區塊。這對於索引資料和將其傳遞到模型都很有用，因為大塊更難搜尋並且不適合模型的有限上下文視窗。

* 存儲：我們需要某個地方來儲存和索引我們的分割，以便以後可以搜尋它們。這通常是使用 [VectorStore](https://python.langchain.com/docs/modules/data_connection/vectorstores/) 和 [Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/) 模型來完成的


### Retrieval and generation 檢索和生成

Retrieve: Given a user input, relevant splits are retrieved from storage using a Retriever.
Generate: A ChatModel / LLM produces an answer using a prompt that includes the question and the retrieved data

* 檢索 ：給定使用者輸入，使用 [檢索器Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/) 從儲存體中擷取相關的分割 。
* 生成：[ChatModel](https://python.langchain.com/docs/modules/model_io/chat/) / [LLM](https://python.langchain.com/docs/modules/model_io/llms/)使用包含問題和檢索到的資料的提示產生答案

## Setup 設定

### 所需套件 Dependencies

在本演示中，我們將使用 OpenAI 聊天模型,嵌入以及 Chroma 向量存儲，但此處顯示的所有內容都適用於任何 [ChatModel](https://python.langchain.com/docs/modules/model_io/chat/)或 [LLM](https://python.langchain.com/docs/modules/model_io/llms/)、 [Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/)以及 [VectorStore](https://python.langchain.com/docs/modules/data_connection/vectorstores/)或 [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/)。

我們將使用以下套件：

In [8]:
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

我們需要OPENAI_API_KEY為嵌入模型設定環境變量，可以直接完成或從.env文件加載，如下所示：



In [9]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# for LLM model
os.environ["OPENAI_API_KEY"] = "填入你的OpenAI API KEY"

# import dotenv

# dotenv.load_dotenv()

### LangSmith

您使用 LangChain 建立的許多應用程式將包含多個步驟以及多次呼叫 LLM 調用。隨著這些應用程式變得越來越複雜，能夠檢查鍊或代理內部到底發生了什麼變得至關重要。最好的方法是使用[LangSmith](https://smith.langchain.com/)。

請注意，LangSmith 不是必需的，但它很有幫助。如果您確實想使用 LangSmith，請在上方的連結註冊後，請確保設定環境變數以開始記錄追蹤：

In [10]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_API_KEY"] = "填入你的LangSmith API KEY(非必要)"

## Preview 預覽

在本指南中，我們將基於Lilian Weng 的[LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/)部落格文章建立一個 QA 應用程序，該應用程式允許我們提出有關帖子內容的問題。

我們可以建立一個簡單的索引管道和 RAG 鏈，用大約 20 行程式碼來完成此操作：

In [11]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

接下來會先使用 Open AI 的 API作為演示，[假如需要使用 Anthropic,Google, Cohere ,FireworksAI ,MistralAI,TogetherAI 的 API的話，請移至官網的範例並修改下列的程式碼即可。](https://python.langchain.com/docs/use_cases/question_answering/quickstart/#setup)，區別不會跟下列的程式碼有太多差異。

In [12]:
!pip install -qU langchain-openai

In [15]:
import getpass
import os
from langchain_openai import ChatOpenAI

#os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = "填入你的OpenAI API KEY"

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [16]:
# Load, chunk and index the contents of the blog.
# 讀取資料，轉為向量儲存
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
# 進行檢索與生成
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("What is Task Decomposition? 請以繁體中文回答")

'任務分解是將問題分解為多個思考步驟，每個步驟生成多個思考，創建一個樹結構。可以通過簡單提示、任務特定指令或人類輸入來進行任務分解。這種技術可以將大型任務轉化為多個易於管理的任務，並揭示模型思考過程的解釋。'

In [18]:
rag_chain.invoke("能把我說下這篇文章大致的內容嗎？ 請以繁體中文回答")

'這篇文章是關於編寫程式碼的指示，包括步驟和要求。文章要求將架構的細節都實現為程式碼。需要按照命名慣例編寫文件，包括所有的類、函數、方法等。'

In [19]:
# cleanup 清除
vectorstore.delete_collection()

可查閱 [LangSmith trace](https://smith.langchain.com/public/1c6ca97e-445b-4d00-84b4-c7befcbc59fe/r/0e1ca4f7-edd6-4cf2-90fd-558dfe2175e5)

## 詳細的流程 Detailed walkthrough

讓我們逐步瀏覽上面的程式碼，以真正了解發生了什麼。

### 1. Indexing: Load 索引：載入

我們需要先載入部落格文章內容。為此，我們可以使用 DocumentLoaders ，它們是從來源載入資料並傳回 Documents清單的物件。 Document 是一個帶有一些page_content(str) 和 metadata(dict)的物件。

在本例中，我們將使用 WebBaseLoader，它用於urllib從 Web URL 載入 HTML 並用BeautifulSoup將其解析為文字。我們可以透過 bs_kwargs 向BeautifulSoup解析器傳遞參數來自訂 HTML - >文字解析（請參閱 BeautifulSoup 文件）。在這種情況下，只有「post-content」、「post-title」或「post-header」類別的 HTML 標記是相關的，因此我們將刪除所有其他標記。

In [20]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [21]:
len(docs[0].page_content)

43131

In [22]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


#### Go deeper 深入了解
DocumentLoader: 從來源載入資料作為 Documents.

[Docs 文件](https://python.langchain.com/docs/modules/data_connection/document_loaders/): 有關如何使用 DocumentLoaders 的詳細文件 .

[Integrations 整合集](https://python.langchain.com/docs/integrations/document_loaders/): 160 多個整合集可供選擇。

[Interface 介面](https://api.python.langchain.com/en/latest/document_loaders/langchain_core.document_loaders.base.BaseLoader.html): 基本介面的 API 參考。




### 2. Indexing: Split 索引:拆分

我們載入的文檔長度超過 42k 個字元。這太長了以至於無法適應許多模型的上下文視窗。即使對於那些可以在其上下文視窗中容納完整帖子的模型，模型也可能很難在很長的輸入中找到資訊。

為了解決這個問題，我們將把它們分成Document塊以進行嵌入和向量儲存。這應該可以幫助我們在運行時僅檢索部落格文章中最相關的部分。

在本例中，我們將把文件分成 1000 個字元的區塊，區塊之間有 200 個字元的重疊。重疊有助於降低將語句與與其相關的重要上下文分開的可能性。我們使用 [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter/)，它將使用常見的分隔符號（例如換行符）遞歸地分割文檔，直到每個區塊的大小合適。這是針對一般文字用例建議的文字分割器。

我們設定add_start_index=True為將初始文件中每個分割文件開始的字元索引保留為元資料屬性「start_index」。

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [24]:
len(all_splits)

66

In [25]:
len(all_splits[0].page_content)

969

In [26]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

#### Go deeper 深入暸解

TextSplitter: 將文件清單分割成更小的區塊的物件。 此為 DocumentTransformers 的子類別。

*   探索上下文感知分割器(Context-aware splitters)，它保留原始文件中每個分割的位置（“上下文或關聯”）： - [Markdown 文件](https://python.langchain.com/docs/modules/data_connection/document_transformers/markdown_header_metadata/)
*   [程式碼（py或js）](https://python.langchain.com/docs/integrations/document_loaders/source_code/)
*   [學術的論文](https://python.langchain.com/docs/integrations/document_loaders/grobid/)
*   [介面：](https://api.python.langchain.com/en/latest/base/langchain_text_splitters.base.TextSplitter.html)基本介面的API參考。

DocumentTransformer：對文件清單執行轉換的物件。

*   [文件：](https://python.langchain.com/docs/modules/data_connection/document_transformers/)有關如何使用 DocumentTransformers 的詳細文檔
*   [整合集](https://python.langchain.com/docs/integrations/document_transformers/)
*   [介面：](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.transformers.BaseDocumentTransformer.html)基本介面的API參考。

### 3. Indexing: Store 索引：存儲

現在我們需要為 66 個文字區塊建立索引，以便我們可以在運行時搜尋它們。最常見的方法是嵌入每個文件分割的內容並將這些嵌入插入向量資料庫（或向量儲存）中。當我們想要搜尋分割時，我們採用文字搜尋查詢，將其嵌入，並執行某種「相似性」搜索，以識別與查詢嵌入最相似的嵌入的儲存分割。最簡單的相似度量是餘弦相似性——我們測量每對嵌入（高維向量）之間角度的餘弦。

[我們可以使用Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/)向量儲存和 [OpenAIEmbeddings](https://python.langchain.com/docs/integrations/text_embedding/openai/)模型將所有文件分割嵌入並儲存在單一命令中 。


In [27]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

#### Go deeper 深入了解

Embeddings: 文字嵌入模型的包裝，用於將文字轉換為嵌入向量。

* [文件](https://python.langchain.com/docs/modules/data_connection/text_embedding/): 有關如何使用嵌入的詳細文件。
* [整合集](https://python.langchain.com/docs/integrations/text_embedding/): 30 多個整合可供選擇。
* [介面](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.Embeddings.html): 基本介面的 API 參考。

VectorStore: 向量資料庫的包裝，用於儲存和查詢嵌入。

* [文件](https://python.langchain.com/docs/modules/data_connection/vectorstores/): 有關如何使用向量儲存的詳細文件。
* [整合集](https://python.langchain.com/docs/integrations/vectorstores/): 40 多個整合可供選擇。
* [介面](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html): 基本介面的 API 參考。

這樣就完成了管道的索引部分。此時，我們有一個可查詢的向量存儲，其中包含我們部落格文章的分塊內容。給定用戶問題，理想情況下我們應該能夠返回回答該問題的部落格文章的片段。

### 4. Retrieval and Generation: Retrieve 檢索與生成：檢索

現在讓我們來編寫實際的應用程式邏輯。我們想要創建一個簡單的應用程序，它接受用戶問題，搜尋與該問題相關的文檔，將檢索到的文檔和初始問題傳遞給模型，然後返回答案。

首先，我們需要定義搜尋文檔的邏輯。 LangChain 定義了一個 [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/)接口，它包裝了一個索引，可以根據給定的字串查詢返回相關的索引Documents。

最常見的類型Retriever是 [VectorStoreRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore/)，它使用向量儲存的相似性搜尋功能來增強檢索。 AnyVectorStore可以很容易地變成 Retrieverwith VectorStore.as_retriever()：



In [28]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition? 請以繁體中文回答")
len(retrieved_docs)

6

In [29]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


#### Go deeper 深入了解

向量儲存通常用於檢索，但也有其他方式可進行檢索。

Retriever：給定文字查詢傳回 Documents 的對象

* [文件](https://python.langchain.com/docs/modules/data_connection/retrievers/): 有關介面和內建檢索技術的更多文件。其中一些包括：
  * MultiQueryRetriever [產生輸入問題的變體](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever/) 以提高檢索命中率。
  * MultiVectorRetriever [以圖表流程取代生成多個問體的變體](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector/)以提高檢索命中率。
  * Max marginal relevance 選擇檢索文件之間的[相關性和多樣性](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf) ，以避免傳遞重複的上下文。
  * 文件可以在向量儲存檢索期間使用元資料過濾器（例如使用[Self Query Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/) ）來過濾文件。
* [整合集](https://python.langchain.com/docs/integrations/retrievers/)：與檢索服務整合。
* [介面](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html)：基本介面的 API 參考。


### 5. Retrieval and Generation: Generate 檢索與生成：生成

讓我們將所有內容放在一個鏈中，該鏈接受問題、檢索相關文件、建構提示、將其傳遞給模型並解決輸出。

我們將使用 gpt-3.5-turbo OpenAI 聊天模型，[但任何 LangChainLLM 或ChatModel都可以替換。](https://python.langchain.com/docs/use_cases/question_answering/quickstart/#retrieval-and-generation-generate)

In [30]:
#
!pip install -qU langchain-openai

In [31]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain import hub

#os.environ["OPENAI_API_KEY"] = getpass.getpass()

os.environ["OPENAI_API_KEY"] =  "填入你的OpenAI API KEY"

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 我們將使用已放入 LangChain prompt hub（https://smith.langchain.com/hub/rlm/rag-prompt）的 RAG 提示。


prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [32]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


我們將使用[LCEL Runnable](https://python.langchain.com/docs/expression_language/) 協定來定義鏈，使我們能夠 - 以透明的方式將元件和函數連接在一起 - 在 LangSmith 中自動追蹤我們的鏈 - 開箱即用地進行串流、非同步和批量調用

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
for chunk in rag_chain.stream("What is Task Decomposition? 請分別用英文和繁體中文回答"):
    print(chunk, end="", flush=True)

Task Decomposition is a technique that breaks down a problem into multiple steps, generating a tree structure for exploration. 任務分解是一種將問題拆分為多個步驟的技術，並生成一個樹狀結構進行探索。

查看 [LangSmith trace](https://smith.langchain.com/public/da23c4d8-3b33-47fd-84df-a3a582eedf84/r)

### Next steps 下一步

我們在很短的時間內涵蓋了很多內容。上述每個部分都有大量的功能、整合和擴展可供探索。除了上面提到的深入來源之外，下一步的良好步驟包括：

* Return sources: 了解如何返回來源文檔
* Streaming: 了解如何串流輸出和中間處理的步驟
* Add chat history: 了解如何將聊天記錄新增至您的應用程式


In [35]:
import os

# for LLM model
os.environ["OPENAI_API_KEY"] =  "填入你的OpenAI API KEY"

# for langSmith (可以不使用)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] =  "填入你的LangSmith API KEY(非必要)"

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 讀取資料，轉為向量儲存
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
print('完成向量化')


# 進行檢索與生成
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("完成建立RAG")

result = rag_chain.invoke("What is Task Decomposition? 請用繁體中文回答。")
print(result)

完成向量化
完成建立RAG
任務分解是將問題分解為多個思考步驟，每個步驟生成多個想法，創建樹狀結構。可以通過簡單提示、任務特定指令或人類輸入來進行任務分解。CoT將大型任務轉換為多個可管理的任務，並揭示模型思考過程的解釋。


In [36]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is. """
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """你是問答任務的助手。 \
使用以下檢索到的上下文來回答問題。 \
如果你不知道答案，就說你不知道。 \
最多使用三個句子並保持答案簡潔\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [38]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition? 請用繁體中文回答。"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What are common ways of doing it? 請用繁體中文回答。"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

常見的任務分解方法包括分層任務分解、模塊化任務分解和分散式任務分解。這些方法可以幫助將複雜任務分解為更小的子任務，以便模型更有效地處理。
